In [1]:
import pandas as pd
from pathlib import Path
import re

In [2]:
PATH_ONE_DRIVE = Path(r'C:\Users\h-pgy\one_drive_prefs\OneDrive - Default Directory\Shared Documents\Estruturação do PDM 2021-2024')

In [3]:
def open_planilha(path_arquivo = None):
    
    if path_arquivo is None:
        path_arquivo = 'Sistematização\Devolutivas Planilhas Secretarias - Final\_Devolutiva - Base de dados V3 - USAR ESTE.xlsx'
    
    path_arquivo = PATH_ONE_DRIVE/path_arquivo
    
    return pd.read_excel(path_arquivo, sheet_name = 'resposta')

In [4]:
def open_controle(path_arquivo = None):
    
    if path_arquivo is None:
        path_arquivo = 'Elaboração PDM Versão Final\Controle das Devolutivas.xlsx'
    
    path_arquivo = PATH_ONE_DRIVE/path_arquivo
    
    return pd.read_excel(path_arquivo)

In [5]:
def convert_float(item):
    
    if pd.isnull(item):
        return item
    
    str_item = str(item)
    
    if ',' in str_item:
        str_item = str_item.replace(',', '.')
        return float(str_item)
    if '.' in str_item:
        return float(str_item)
    try:
        return int(item)
    except ValueError:
        pass
    
    return item

def pegar_novo_numero(meta_citada, controle):
    
    row_controle = controle[controle['Meta']==meta_citada]
    
    if not row_controle.empty:
        row = row_controle.iloc[0]
        return row['NOVO Nº Meta']
    else:
        print(f'Meta {meta_citada} não encontrada no controle!')
        return meta_citada

def clean_float(item):
    
    item = str(item)
    
    if '.' in item:
        esquerda, direita = item.split('.')
        if int(direita)>0:
            return item
        else:
            return esquerda
    
    return item

def alterar_meta_resposta(row, planilha_controle):
    
    resp_inicial = row['Resposta inicial']
    meta_citada = row['meta_original_citada']
    if pd.notnull(meta_citada):
        
        meta_citada_float = convert_float(meta_citada)
        nova_meta = pegar_novo_numero(meta_citada_float, 
                                      planilha_controle)
        nova_meta = clean_float(nova_meta)
        pat = f'#Meta {meta_citada}#'
        resp_alterada = resp_inicial.replace(pat, f'Meta {nova_meta}')
        
        return resp_alterada
    return ''

def segunda_alteracao(row, planilha_controle):
    
    resp_inicial = row['Resposta alterada H']
    meta_citada = row['segunda_meta_citada']
    if pd.notnull(meta_citada):
        
        meta_citada_float = convert_float(meta_citada)
        nova_meta = pegar_novo_numero(meta_citada_float, 
                                      planilha_controle)
        nova_meta = clean_float(nova_meta)
        pat = f'#Meta {meta_citada}#'
        resp_alterada = resp_inicial.replace(pat, f'Meta {nova_meta}')
        
        return resp_alterada
    return resp_inicial

In [6]:
def alterar_respostas(df, controle):
    
    df = df.copy()
    df['Resposta inicial'] = df['Resposta inicial'].fillna('')
    
    pat = re.compile(r'#[mM]eta (\d{1,2},{0,1}\d{0,1})#')
    df['meta_original_citada'] = df['Resposta inicial'].str.extract(pat)
    print('Primeiras metas citadas')
    print(df['meta_original_citada'].unique())
    df['Resposta alterada H'] = df.apply(alterar_meta_resposta, args = [controle], axis = 1)
    df['segunda_meta_citada'] = df['Resposta alterada H'].str.extract(pat)
    print('Segundas metas citadas')
    print(df['segunda_meta_citada'].unique())
    df['Resposta alterada H'] = df.apply(segunda_alteracao, args = [controle], axis = 1)
    df['terceira_meta_citada'] = df['Resposta alterada H'].str.extract(pat)
    print('Terceiras metas citadas - o que aparecer aqui nao sera alterado')
    print(df['terceira_meta_citada'].unique())
    
    return df

In [7]:
controle = open_controle()

In [8]:
df = open_planilha()

In [9]:
df = alterar_respostas(df, controle)

Primeiras metas citadas
[nan '75' '52' '50' '17' '51' '49' '54' '57' '19' '55' '39' '38' '27' '28'
 '30' '14' '16' '2' '47']
Segundas metas citadas
[nan '56' '51' '29' '31']
Terceiras metas citadas - o que aparecer aqui nao sera alterado
[nan]


In [10]:
def tipo_vinculo(dado, controle):
    
    dado_str = str(dado)
    pat_num = '\d{1,2}'
    pat_letra = '[A-Z]{1}'
    
    match_num = re.match(pat_num, dado_str)
    match_letra = re.match(pat_letra, dado_str)
    if match_num and match_num.group()==dado_str:
        return 'meta'
    elif match_letra and match_letra.group()==dado_str:
        return 'objetivo'
    else:
        return 'nenhum'
    
def convert_float(item):
    
    if pd.isnull(item):
        return item
    
    str_item = str(item)
    
    if ',' in str_item:
        str_item = str_item.replace(',', '.')
        return float(str_item)
    if '.' in str_item:
        return float(str_item)
    try:
        return int(item)
    except ValueError:
        pass
    
    return item

def find_nova_meta(meta_antiga, controle):
    
    meta_antiga = convert_float(meta_antiga)
    if meta_antiga == 5:
        meta_antiga = 5.1
    row_controle = controle[controle['Meta']==meta_antiga]
    
    if not row_controle.empty:
        row = row_controle.iloc[0]
        return row['NOVO Nº Meta']
    else:
        print(f'Meta {meta_antiga} não encontrada no controle!')
        return meta_antiga
    

def find_novo_objetivo(objetivo_antigo, controle):
    
    row_controle = controle[controle['Objetivo Antigo']==objetivo_antigo]
    
    if not row_controle.empty:
        row = row_controle.iloc[0]
        return row['Objetivo Estratégico']
    else:
        print(f'Meta {meta_antiga} não encontrada no controle!')
        return meta_antiga
    
def autualizar_vinculo(vinculo_antigo, controle):
    
    tipo = tipo_vinculo(vinculo_antigo, controle)
    
    if tipo == 'meta':
        return find_nova_meta(vinculo_antigo, controle)
    
    elif tipo == 'objetivo':
        return find_novo_objetivo(vinculo_antigo, controle)
    
    elif tipo == 'nenhum':
        return vinculo_antigo
    else:
        print(tipo)
        raise ValueError
        
def find_eixo_meta(meta, controle):
    
    row = controle[controle['NOVO Nº Meta']==meta]
    
    if not row.empty:
    
        return row['Eixo'].iloc[0]
    else:
        print(f'Meta {meta} nao encontrado')
    
def find_eixo_objetivo(objetivo, controle):
    
    row = controle[controle['Objetivo Estratégico']==objetivo]
    
    if not row.empty:
    
        return row['Eixo'].iloc[0]
    else:
        print(f'Objetivo {objetivo} nao encontrado')
        
def pegar_eixo(row, controle):
    
    tipo = tipo_vinculo(row['Vinculo'], controle)
    vinculo_novo = row['Vinculo_novo']
    if tipo == 'meta':
        return find_eixo_meta(vinculo_novo, controle)
    
    elif tipo == 'objetivo':
        return find_eixo_objetivo(vinculo_novo, controle)
    
    elif tipo == 'nenhum':
        return 'N/A'
    else:
        print(tipo)
        raise ValueError 

In [11]:
df['Vinculo_novo'] = df['Vinculo'].apply(autualizar_vinculo, args=[controle])

In [12]:
df['Vinculo_eixo'] = df.apply(pegar_eixo, args=[controle], axis = 1)

In [13]:
df['Vinculo_eixo'].unique()

array(['N/A', 'SP Justa e Inclusiva_x000D_', 'SP Global e \nSustentável',
       'SP Eficiente', 'SP Inovadora e \nCriativa_x000D_', 'SP Ágil ',
       'SP Segura e Bem \nCuidada '], dtype=object)

In [14]:
df.to_excel('devolutivas_arrumado_com_eixo.xlsx')